In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128 #128 - original batch size
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
#print (img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 54s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 32  
# original is 12 
# 64, 128 and 256 are giving above 1MM+ Parameters, hence going with 32.
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 32)   128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 32)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(),
              metrics=['accuracy'])

In [13]:
# Adding Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.1, zoom_range=0.1)

test_gen = ImageDataGenerator()

train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
test_generator = test_gen.flow(x_test, y_test, batch_size=batch_size)


model.fit_generator(train_generator, steps_per_epoch=60000//batch_size, epochs=epochs, validation_data=test_generator, validation_steps=10000//batch_size)

Epoch 1/50
468/468 [==============================] - 291s 621ms/step - loss: 2.1042 - acc: 0.2025 - val_loss: 2.0124 - val_acc: 0.2237
Epoch 2/50
468/468 [==============================] - 270s 577ms/step - loss: 1.7542 - acc: 0.3196 - val_loss: 1.7005 - val_acc: 0.3616
Epoch 3/50
468/468 [==============================] - 270s 578ms/step - loss: 1.6391 - acc: 0.3686 - val_loss: 1.6291 - val_acc: 0.3938
Epoch 4/50
468/468 [==============================] - 270s 577ms/step - loss: 1.5547 - acc: 0.4088 - val_loss: 2.2953 - val_acc: 0.3057
Epoch 5/50
468/468 [==============================] - 270s 577ms/step - loss: 1.4824 - acc: 0.4444 - val_loss: 2.0470 - val_acc: 0.3864
Epoch 6/50
468/468 [==============================] - 271s 579ms/step - loss: 1.4147 - acc: 0.4761 - val_loss: 1.5065 - val_acc: 0.4829
Epoch 7/50
468/468 [==============================] - 271s 578ms/step - loss: 1.3492 - acc: 0.5010 - val_loss: 2.0909 - val_acc: 0.4020
Epoch 8/50
468/468 [============================

In [14]:
# Model fit implimentation that came with original code
# Runnig for 50 epochs
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 227s 5ms/step - loss: 0.5965 - acc: 0.7916 - val_loss: 0.9059 - val_acc: 0.7202
Epoch 2/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.5877 - acc: 0.7958 - val_loss: 0.8832 - val_acc: 0.7386
Epoch 3/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.5829 - acc: 0.7989 - val_loss: 0.7369 - val_acc: 0.7754
Epoch 4/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.5710 - acc: 0.8034 - val_loss: 0.8572 - val_acc: 0.7519
Epoch 5/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.5698 - acc: 0.8022 - val_loss: 0.8190 - val_acc: 0.7522
Epoch 6/50
50000/50000 [==============================] - 226s 5ms/step - loss: 0.5647 - acc: 0.8045 - val_loss: 0.8262 - val_acc: 0.7562
Epoch 7/50
50000/50000 [==============================] - 225s 4ms/step - loss: 0.5581 - acc: 0.8059 - val_loss: 0.7839 - 

In [15]:
# Running for 25 epochs
model.fit_generator(train_generator, steps_per_epoch=60000//batch_size, epochs=epochs//2, validation_data=test_generator, validation_steps=10000//batch_size)

Epoch 1/25
468/468 [==============================] - 268s 573ms/step - loss: 0.5027 - acc: 0.8271 - val_loss: 0.7950 - val_acc: 0.7809
Epoch 2/25
468/468 [==============================] - 268s 573ms/step - loss: 0.5016 - acc: 0.8292 - val_loss: 0.7699 - val_acc: 0.7894
Epoch 3/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4847 - acc: 0.8317 - val_loss: 0.6935 - val_acc: 0.8021
Epoch 4/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4812 - acc: 0.8325 - val_loss: 0.7994 - val_acc: 0.7820
Epoch 5/25
468/468 [==============================] - 268s 573ms/step - loss: 0.4766 - acc: 0.8355 - val_loss: 0.7693 - val_acc: 0.7950
Epoch 6/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4777 - acc: 0.8359 - val_loss: 0.8063 - val_acc: 0.7800
Epoch 7/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4666 - acc: 0.8400 - val_loss: 0.7514 - val_acc: 0.7926
Epoch 8/25
468/468 [============================

In [16]:
# adding another training for 50 epochs
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.3378 - acc: 0.8847 - val_loss: 0.5719 - val_acc: 0.8346
Epoch 2/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.3303 - acc: 0.8880 - val_loss: 0.5412 - val_acc: 0.8457
Epoch 3/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.3224 - acc: 0.8895 - val_loss: 0.6107 - val_acc: 0.8321
Epoch 4/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.3230 - acc: 0.8892 - val_loss: 0.5872 - val_acc: 0.8368
Epoch 5/50
50000/50000 [==============================] - 225s 4ms/step - loss: 0.3206 - acc: 0.8898 - val_loss: 0.5880 - val_acc: 0.8397
Epoch 6/50
50000/50000 [==============================] - 225s 4ms/step - loss: 0.3149 - acc: 0.8914 - val_loss: 0.6703 - val_acc: 0.8229
Epoch 7/50
50000/50000 [==============================] - 225s 5ms/step - loss: 0.3166 - acc: 0.8916 - val_loss: 0.5569 - 

In [1]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

NameError: ignored

In [22]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:

from google.colab import files

files.download('DNST_model.h5')